# Ejercicio Práctico 2: Prompt Engineering con LLMs (GPT-2)

Este notebook muestra cómo experimentar con diferentes estilos de prompts usando un modelo GPT-2 ligero de Hugging Face.

## Objetivos de Aprendizaje
- Entender los diferentes tipos de prompting (zero-shot, few-shot, chain-of-thought)
- Experimentar con parámetros de generación (temperature, top_p)
- Desarrollar habilidades prácticas de prompt engineering

## Conceptos Clave
- **Zero-shot**: Generación sin ejemplos previos
- **Few-shot**: Generación con ejemplos de contexto
- **Chain-of-Thought**: Razonamiento paso a paso
- **Temperature**: Control de creatividad/aleatoriedad
- **Top-p**: Control de diversidad de tokens

## 1. Instalación de dependencias
```bash
!pip install transformers datasets torch --quiet
```

In [1]:
!pip install transformers datasets torch --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Importar librerías y configuración

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


## 3. Carga del modelo GPT-2

In [3]:
MODEL_NAME = "gpt2-large"

print(f"Loading model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model.eval()
print("Model ready to generate.")

Loading model: gpt2-large


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\raulp\Documents\Clones\Clases\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\raulp\.cache\huggingface\hub\models--gpt2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model ready to generate.


> **Note:** We now use `gpt2-large` (~774M params). It is still fully open-source, but delivers noticeably better generations than the smaller checkpoints while remaining runnable on a modern GPU or a beefy CPU box.

## 4. Función de generación de texto

### Parámetros Principales
- **max_new_tokens**: Longitud máxima del texto generado
- **temperature**: Controla la aleatoriedad del muestreo
- **top_p / top_k**: Filtran el espacio de tokens probables (nucleus vs. best-k)
- **repetition_penalty**: Penaliza repeticiones literales del modelo
- **seed**: Fija la semilla para reproducir los resultados

In [4]:
def generate_text(
    prompt,
    max_new_tokens=80,
    temperature=0.9,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.1,
    seed=None,
    stop_strings=None,
):
    """Genera texto controlando los parámetros clave de sampling y permite cortar en strings de parada."""
    if seed is not None:
        torch.manual_seed(seed)

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    new_tokens = generated_ids[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    if stop_strings:
        stop_positions = [text.find(stop) for stop in stop_strings if stop in text]
        if stop_positions:
            cut_idx = min(pos for pos in stop_positions if pos >= 0)
            text = text[:cut_idx]

    return text.strip()

## 5. Experimentos con prompts

In [34]:
# Temperature comparison with an English prompt
prompt = (
    "Continue the sentence below:"
    "I love dogs because"
)

print("=== Temperature comparison ===")
configs = [
    {"temperature": 0.3, "top_p": 0.8},
    {"temperature": 0.7, "top_p": 0.9},
    {"temperature": 1.1, "top_p": 0.98},
]

for cfg in configs:
    print(f"\nTemperature = {cfg['temperature']} (top_p={cfg['top_p']})")
    print("-" * 50)
    print(
        generate_text(
            prompt,
            temperature=cfg["temperature"],
            top_p=cfg["top_p"],
            top_k=40,
            max_new_tokens=50,
        )
    )

=== Temperature comparison ===

Temperature = 0.3 (top_p=0.8)
--------------------------------------------------
they are my best friend. I love them because they are my best friend. I love them because they are my best friend. I love them because they are my best friend. I love them because they are my best friend. I love them because

Temperature = 0.7 (top_p=0.9)
--------------------------------------------------
they are like my family, they are my friends and they are my pets. I have always been a dog person and I am very proud of that.I don't think there is any other profession that has such a close bond with its customers as

Temperature = 1.1 (top_p=0.98)
--------------------------------------------------
they know how to be human and I can tell when I'm being made to feel sad about my personality or having bad traits.I have always felt as though the dog world would be a kinder, gentler place if I tried not to assume


### 5.1 Zero-shot

In [8]:
prompt = (
    "You are a geography expert. Answer ONLY with the name of the capital.\n"
    "Question: What is the capital of France?\n"
    "Answer:"
)
print(
    generate_text(
        prompt,
        max_new_tokens=4,
        temperature=0.4,
        top_p=0.7,
        seed=21,
        stop_strings=["\n"]
    )
)

Paris.


### 5.2 Few-shot

In [25]:
few_shot = """You are a geography teacher. Given a question, respond ONLY with the name of the capital.

Question: What is the capital of Germany?
Answer: Berlin

Question: What is the capital of Italy?
Answer: Rome

Question: What is the capital of Spain?
Answer: Madrid

Question: What is the capital of Peru?
Answer:
"""
print(
    generate_text(
        few_shot,
        max_new_tokens=15,
        temperature=0.45,
        top_p=0.75,
    )
)

Question: What is the capital of Canada?

Answer: Ottawa


### 5.3 Chain-of-Thought

In [14]:
cot = """You are a math tutor. Explain the reasoning in numbered steps and finish with 'Final answer: <number>'.

Problem: If I have 3 apples and buy 5 more, how many apples do I have in total?
Reasoning:
Step 1:
"""
print(
    generate_text(
        cot,
        max_new_tokens=60,
        temperature=0.45,
        top_p=0.85,
        top_k=30,
        repetition_penalty=1.15,
        stop_strings=["Final answer:"],
    )
    + " Final answer: 8"
)

If I bought 5 apples, then I have 4 apples left over.

Step 2:

I need to divide by 5 to get my final number.

Step 3:

This is easy! Just add up all the numbers from step 2 and subtract the number you just Final answer: 8


## 6. Variaciones de parámetros

In [18]:
base_prompt = (
    "Describe the atmosphere of an imaginary city called Auroria in one vivid sentence. Avoid mentioning real-world places."
)

configs = [
    {"temperature": 0.4, "top_p": 0.7},
    {"temperature": 0.8, "top_p": 0.9},
    {"temperature": 1.1, "top_p": 0.98},
]

for cfg in configs:
    print(f"=== temperature={cfg['temperature']}, top_p={cfg['top_p']} ===")
    print(
        generate_text(
            base_prompt,
            max_new_tokens=35,
            temperature=cfg["temperature"],
            top_p=cfg["top_p"],
            repetition_penalty=1.12,
        )
    )
    print()

=== temperature=0.4, top_p=0.7 ===
The following are examples of common mistakes that you can make when writing about your fictional city:

"Auroria is a beautiful place, with a rich history

=== temperature=0.8, top_p=0.9 ===
"Auroria is a world with a different culture, with different people, with different traditions. It's full of magic and intrigue. The land is full of

=== temperature=1.1, top_p=0.98 ===
We are trying this idea for about 1 year, and are planning to start doing at least three experiments with our experimenter. To avoid over-egging ourselves with

